In [1]:
from datasets import load_dataset

dataset = load_dataset("agentica-org/DeepScaleR-Preview-Dataset",
                       trust_remote_code=True,
                       split='train')

/mbz/users/shibo.hao/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import json

output_count = json.load(open("question_output_deepscale_r.json"))

In [9]:
output_count["0"]  # {"output": "count"}

{'-\\dfrac{2}{3}': 64}

In [6]:
from verl.utils.reward_score.math_llm_judge import grade_answer
from verl.utils.reward_score.math_llm_judge import math_equal
from verl.utils.reward_score.math_llm_judge import llm_check_answer
import math


2025-03-20 17:19:11,246	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [10]:
def compute_score_with_output(extracted_model_output, ground_truth, question):
    # grade simple algebra questions. if succeeded, return; otherwise, proceed to more complex grading
    if grade_answer(extracted_model_output, ground_truth):
        return True, True, extracted_model_output

    try:
        if "\\pi" in extracted_model_output or "\\pi" in ground_truth:
            equivs = []
            for pi in [math.pi, 3.14]:
                equivs.append(math_equal(extracted_model_output, ground_truth, timeout=True, pi=pi))
            is_correct = any(equivs)
        else:
            is_correct = math_equal(extracted_model_output, ground_truth, timeout=True)
    except:
        is_correct = False
        
    if is_matched and not is_correct:
        # use llm to check if the answer is correct
        try:
            is_correct = int(ground_truth) == int(extracted_model_output)
        except:
            is_correct = llm_check_answer(extracted_model_output, ground_truth, question)

    return is_correct, 1, extracted_model_output


In [32]:
correct = 0
total = 0
stats = {}

for question_id, outputs in tqdm(output_count.items()):
    question_id = int(question_id)
    saved_outputs = []
    for output, count in outputs.items():
        if output == "None":
            saved_outputs.append(("None", count, False))
        else:
            is_correct, _, _ = compute_score_with_output(output, answers[question_id], questions[question_id])
            saved_outputs.append((output, count, is_correct))
    stats[question_id] = saved_outputs


  2%|▏         | 8/512 [00:00<00:46, 10.73it/s]

{'model_output': '\\dfrac{5}{2}', 'ground_truth': '\\frac{5}{3}', 'question': 'Points $A,B,C,D,E$ and $F$ lie, in that order, on $\\overline{AF}$, dividing it into five segments, each of length 1. Point $G$ is not on line $AF$. Point $H$ lies on $\\overline{GD}$, and point $J$ lies on $\\overline{GF}$. The line segments $\\overline{HC}, \\overline{JE},$ and $\\overline{AG}$ are parallel. Find $HC/JE$.', 'response': "The student's answer is \\(\\dfrac{5}{2}\\), whereas the reference answer is \\(\\frac{5}{3}\\). These two fractions are not mathematically equivalent, as \\(\\dfrac{5}{2}\\) equals 2.5 and \\(\\frac{5}{3}\\) equals approximately 1.67. Since they do not match, the student's answer is incorrect.\n\nFinal Grade: INCORRECT", 'eval_result': False}
{'model_output': 'x = -\\log_{10} y', 'ground_truth': '10^{-x}', 'question': "The graph, $G$ of $y=\\log_{10}x$ is rotated $90^{\\circ}$ counter-clockwise about the origin to obtain a new graph $G'$. What is the equation for $G'$?", '

  2%|▏         | 10/512 [00:04<04:29,  1.86it/s]

{'model_output': 'Y = 10^{-X}', 'ground_truth': '10^{-x}', 'question': "The graph, $G$ of $y=\\log_{10}x$ is rotated $90^{\\circ}$ counter-clockwise about the origin to obtain a new graph $G'$. What is the equation for $G'$?", 'response': "The reference answer is \\(10^{-x}\\). The student's answer is \\(Y = 10^{-X}\\). These answers are mathematically equivalent, as they both represent the same function. The only difference is the notation for the variables, where the student used uppercase \\(Y\\) and \\(X\\), while the reference answer used lowercase. Since the mathematical content and meaning are the same, the student's answer should be considered correct.\n\nFinal Grade: CORRECT", 'eval_result': True}
{'model_output': '\\dfrac{8\\sqrt{6} - 3}{25}', 'ground_truth': '\\frac{1}{5}', 'question': 'A rectangular yard contains two flower beds in the shape of congruent isosceles right triangles. The remainder of the yard has a trapezoidal shape, as shown. The parallel sides of the trapezo

  2%|▏         | 12/512 [00:06<05:15,  1.59it/s]

...

  3%|▎         | 13/512 [00:06<05:10,  1.61it/s]

{'model_output': '260^\\circ', 'ground_truth': '380', 'question': 'What is the sum of all possible values of $t$ between $0$ and $360$ such that the triangle in the coordinate plane whose vertices are $(\\cos 40^\\circ,\\sin 40^\\circ)$, $(\\cos 60^\\circ,\\sin 60^\\circ)$, and $(\\cos t^\\circ,\\sin t^\\circ)$ is isosceles?', 'response': "The student's answer is $260^\\circ$, while the reference answer provided is $380$. Since the reference answer and the student's answer are not numerically equivalent and there is no indication that the student's answer could be interpreted in a way that matches the reference answer, the student's answer is incorrect based on the criteria given.\n\nFinal Grade: INCORRECT", 'eval_result': False}
{'model_output': '15\\sqrt{3}', 'ground_truth': '20', 'question': 'In the adjoining figure, points $B$ and $C$ lie on line segment $AD$, and $AB, BC$, and $CD$ are diameters of circle $O, N$, and $P$, respectively. Circles $O, N$, and $P$ all have radius $15$ 

  3%|▎         | 13/512 [00:17<11:17,  1.36s/it]

{'model_output': '6\\sqrt{21}', 'ground_truth': '20', 'question': 'In the adjoining figure, points $B$ and $C$ lie on line segment $AD$, and $AB, BC$, and $CD$ are diameters of circle $O, N$, and $P$, respectively. Circles $O, N$, and $P$ all have radius $15$ and the line $AG$ is tangent to circle $P$ at $G$. If $AG$ intersects circle $N$ at points $E$ and $F$, then chord $EF$ has length', 'response': "The reference answer given is \\(20\\), whereas the student's provided answer is \\(6\\sqrt{21}\\). These two values are not equivalent in a mathematical sense. Therefore, the student's answer does not match the reference answer.\n\nFinal Grade: INCORRECT", 'eval_result': False}


KeyboardInterrupt: 

In [ ]:
result_dict = {}

stats = {}

for d in tqdm(data):
    response = d['response']['body']['choices']['message']['content']
    id = d['custom_id']
    question_id = int(id.split("-")[1])
    sample_id = int(id.split("-")[-1])
    if question_id not in stats:
        stats[question_id] = (0, 0)
    
    correct = compute_score(response, answers[question_id])[0]
    stats[question_id] = (stats[question_id][0] + correct, stats[question_id][1] + 1)
    matched, model_output = match_answer(response)
    d['matched'] = matched
    d['model_output'] = model_output
    d['correct'] = correct
    result_dict[f"{question_id}-{sample_id}"] = {
        "matched": matched,
        "model_output": model_output,
        "correct": correct
    }
    